## Load Required Libraries<br>
**Standard Process**
 - ```pip install papermill```<br>
 - ```pip install jupyter```<br> 
 - ```pip install notebook```<br>
 
**Email Process**
 - Same as email_sender
 
**SQL Log Process**
 - Same as da_tran_SQL

# Load setting from Text File

In [1]:
import os, re,json
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

with open('setting.txt','r') as f:
    setting = json.load(f)
    
sql_setting = setting['MSSQL']
sql_setting.keys()

mail_setting = setting['email']
mail_setting.keys()

dict_keys(['user', 'password', 'to', 'server', 'file_path_1', 'file_path_2'])

# Working with run_pipeline 

In [2]:
from py_topping.run_pipeline import run_pipeline

In [ ]:
run_pipeline(script_list = []
             , out_prefix = None
             , out_folder = ''
             , email_dict = None
             , sending = False
             , only_error = False
             , notebook_attached = False
             , attached_only_error = False
             , attached_log = False
             , log_sql = None
             , line_sending = None
             , line_subject = 'Untitled')

### List of Parameter    
- **script_list :** List of Script File to Run (.py or .ipynb)<br>
- **out_prefix :** Prefix of .ipynb output files<br>
- **out_folder :** Folder to contain .ipynb output files<br>
- **email_dict :**  Email config<br>
- **sending :** Send Run Result in Email or Not (True/False)<br>
- **only_error :** Only send email/line when there're error in result (True/False)<br>
- **notebook_attached :** Attached notebook in email or not (True/False)<br>
- **attached_only_error :** Only Attached Error Notebook in email or not (True/False)<br>
- **attached_log :** Attached Result as csv file in email or not (True/False)<br>
- **log_sql :** SQL config<br>
- **line_sending :** lazy_LINE class in py_topping.general_use if you want to send LINE message<br>
- **line_subject :** Subject when send LINE message

**Return :** Pandas DataFrame of Run Result

# Simple Run All Script

In [4]:
run_pipeline(script_list = ['SCRIPT.ipynb', 'SCRIPT.py'])

Executing:   0%|          | 0/1 [00:00<?, ?cell/s]

,start,script,notebook_out,run_result,end
0,2021-12-22 09:54:43,SCRIPT.ipynb,/2021_12_22_09_54_SCRIPT.ipynb,OK,2021-12-22 09:54:47
1,2021-12-22 09:54:47,SCRIPT.py,None,OK,2021-12-22 09:54:47


# Run All Script with Output of .ipynb with customize prefix in specific folder

In [7]:
run_pipeline(script_list = ['SCRIPT.ipynb', 'SCRIPT.py']
             , out_prefix = ''
             , out_folder = 'TEMP')

,start,script,notebook_out,run_result,end
0,2021-02-14 14:29:16,SCRIPT.ipynb,TEMP/SCRIPT.ipynb,OK,2021-02-14 14:29:19
1,2021-02-14 14:29:19,SCRIPT.py,None,OK,2021-02-14 14:29:19


# Run All Script and Send Run Result in Email

## Set Up Mail Config as a dict

In [8]:
mail_dict = {'user' : mail_setting['user']
             , 'password' : mail_setting['password']
             , 'server' : mail_setting['server']
             ,'sendto' : mail_setting['to']
             ,'subject' : 'TEST'}

In [9]:
run_pipeline(['SCRIPT.ipynb', 'SCRIPT.py']
             , email_dict = mail_dict
             , sending = True
             , only_error = False
             , notebook_attached = True
             , attached_only_error = False
             , attached_log = True)

,start,script,notebook_out,run_result,end
0,2021-02-14 14:29:19,SCRIPT.ipynb,/2021_02_14_14_29_SCRIPT.ipynb,OK,2021-02-14 14:29:23
1,2021-02-14 14:29:23,SCRIPT.py,None,OK,2021-02-14 14:29:23


# Run All Script and Log Run Result in SQL

## Set Up SQL Config as a dict

In [10]:
from py_topping.data_connection.database import da_tran_SQL

sql_dict = {'job_name' : 'test_pipeline'
            ,'table_name' : 'test_pipeline'
            , 'da_tran_SQL': da_tran_SQL(sql_type = sql_setting['type']
                                          , host_name = sql_setting['host']
                                          , database_name = sql_setting['database']
                                          , user = sql_setting['user']
                                          , password =  sql_setting['password'])}

Connection OK


In [12]:
run_pipeline(['SCRIPT.ipynb', 'SCRIPT.py']
             , log_sql = sql_dict)
sql_dict['da_tran_SQL'].read('test_pipeline')

,start,script,notebook_out,run_result,end,job_name
0,2021-02-14 14:29:25,SCRIPT.ipynb,/2021_02_14_14_29_SCRIPT.ipynb,OK,2021-02-14 14:29:27,test_pipeline
1,2021-02-14 14:29:27,SCRIPT.py,None,OK,2021-02-14 14:29:27,test_pipeline


# Combine Everything

In [14]:
run_pipeline(['SCRIPT.ipynb', 'SCRIPT.py']
             , out_prefix = 'OUT_'
             , out_folder = 'TEMP'
             , email_dict = mail_dict
             , sending = True
             , only_error = False
             , notebook_attached = True
             , attached_only_error = False
             , attached_log = True
             , log_sql = sql_dict)
sql_dict['da_tran_SQL'].read('test_pipeline')

,start,script,notebook_out,run_result,end,job_name
0,2021-02-14 14:29:27,SCRIPT.ipynb,TEMP/OUT_SCRIPT.ipynb,OK,2021-02-14 14:29:30,test_pipeline
1,2021-02-14 14:29:30,SCRIPT.py,None,OK,2021-02-14 14:29:30,test_pipeline


# New in 0.3.10
# Send Line message

In [4]:
from py_topping.general_use import lazy_LINE

# Init lazy_LINE class
line = lazy_LINE('YOUR LINE NOTI')

# Run
run_pipeline(script_list = ['SCRIPT.ipynb', 'SCRIPT.py']
             , line_sending = line
             , line_subject = 'test krub')

Executing:   0%|          | 0/1 [00:00<?, ?cell/s]

,start,script,notebook_out,run_result,end
0,2021-12-05 15:32:11,SCRIPT.ipynb,/2021_12_05_15_32_SCRIPT.ipynb,OK,2021-12-05 15:32:13
1,2021-12-05 15:32:13,SCRIPT.py,None,OK,2021-12-05 15:32:13
